Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [7]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [8]:
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [54]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [3]:
!pip install category_encoders==2.*

     |████████████████████████████████| 102kB 1.6MB/s ta 0:00:01


In [38]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import category_encoders as ce
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [55]:
# Start with splitting train data into train and validation set

target = 'status_group'

train, val = train_test_split(train, test_size=0.20, random_state=25,
                              stratify=train[target])

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [56]:
# Utilize wrangle function for data cleaning and feature engineering

def wrangle(df):
    """Wrangle train, validate, and test sets in the same way."""
    
    # Create copy to prevent warnings
    df = df.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    df['latitude'] = df['latitude'].replace(-2e-08, 0)
    
    # Some columns have zeros but probably shouldn't, so replace the zeros with 
    # nulls - will impute later with pipeline
    cols_with_zeros = ['longitude', 'latitude','construction_year','gps_height',
                       'population']
    for col in cols_with_zeros:
        df[col] = df[col].replace(0, np.nan)

    # Payment, management, extraction_type, extraction_type_group, source and
    # waterpoint_type all seem redundant, so drop them. Also, id and
    # recorded_by are not useful for this problem.
    df = df.drop(columns=['payment','id','recorded_by'])
    
    # Convert date_recorded to only include year 
    df['date_recorded'] = pd.to_datetime(df['date_recorded'], yearfirst=True)
    df['date_recorded'] = df['date_recorded'].dt.year

    # Construction year looks like it may be relevant, so I'll see if the time 
    # from inspection has a similar correlation to status_group
    df['construction_vs_inspection_years'] = df['construction_year'] - df['date_recorded']

    # 'water_quality' looks interesting as I may expect water sources that produce
    # anything other than "soft" water may have some issues ...
    df['water_is_soft'] = (df['water_quality']=='soft').astype(int)  

    # 'quantity' of the water should tell us something if it's anything other
    # than 'enough'
    df['enough_water'] = (df['quantity']=='enough').astype(int)

    # Noticed that majority of these waterpoints do not have a name, and I would
    # think that a waterpoint built with care/pride would be named. 
    # df['has_name'] = (df['wpt_name']!='none').astype(int)

    
    # return the wrangled dataframe
    return df

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [12]:
train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [57]:
# Define features, x feature matrices and y target vectors

features = train.columns.drop([target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

((47520, 40), (47520,), (11880, 40), (11880,), (14358, 40))

In [14]:
# Baseline prior to cross validation (RandomFOrestClassifier)

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(random_state=25, n_estimators=80, max_depth=31,
                           min_samples_split=11)
)

pipeline.fit(X_train, y_train)

print(f'Train Accuracy(DecisionTree): {pipeline.score(X_train, y_train):%}')
print(f'Validation Accuracy(DecisionTree): {pipeline.score(X_val, y_val):%}')

Train Accuracy(DecisionTree): 91.548822%
Validation Accuracy(DecisionTree): 81.035354%


# Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

In [47]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': [int(x) for x in np.linspace(start=50, stop=400, num=10)], 
    'randomforestclassifier__max_depth': [int(x) for x in np.linspace(start=25, stop=50, num=5)], 
    'randomforestclassifier__max_features': ['auto','log2'],
    'randomforestclassifier__min_samples_split': [1.0,3,5,7,9,11,13,15,17],
    'randomforestclassifier__min_samples_leaf': [1,2,3]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

In [48]:
print('Best hyperparameters', search.best_params_)
print('\n')
print(f'Cross-validation Accuracy: {search.best_score_:%}')

# These results did not improve on the baseline for train, but I'll
# try applying to val and see what I get

Best hyperparameters {'simpleimputer__strategy': 'mean', 'randomforestclassifier__n_estimators': 244, 'randomforestclassifier__min_samples_split': 9, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_depth': 50}


Cross-validation Accuracy: 80.307239%


In [49]:
pipeline = search.best_estimator_
print(f'Validation Accuracy(CrossValidation): {pipeline.score(X_val, y_val):%}')
# So performance on the validation set indicates a (relatively) significant increase

Validation Accuracy(CrossValidation): 81.111111%


#  Submit your predictions to our Kaggle competition.

In [50]:
y_pred = pipeline.predict(X_test)
y_pred

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [52]:
submission = test[['id']].copy()
submission['status_group'] = y_pred
submission

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14353,39307,non functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [53]:
submission.to_csv(r'/Users/ekselan/Downloads/LS/LS_Unit_2/Kaggle_Submissions\tanzania_water_submission_april_15.csv', index=False)

# Try to improve results - increase iterations

In [62]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': [int(x) for x in np.linspace(start=50, stop=400, num=10)], 
    'randomforestclassifier__max_depth': [int(x) for x in np.linspace(start=25, stop=50, num=5)], 
    'randomforestclassifier__max_features': ['auto','log2'],
    'randomforestclassifier__min_samples_split': [1.0,3,5,7,9,11,13,15,17],
    'randomforestclassifier__min_samples_leaf': [1,2,3]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

In [64]:
print('Best hyperparameters', search.best_params_)
print('\n')
print(f'Cross-validation Accuracy: {search.best_score_:%}')
# This clearly made a difference, let's see how it does on validation

Best hyperparameters {'simpleimputer__strategy': 'mean', 'randomforestclassifier__n_estimators': 205, 'randomforestclassifier__min_samples_split': 7, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_depth': 37}


Cross-validation Accuracy: 80.989057%


In [65]:
pipeline = search.best_estimator_
print(f'Validation Accuracy(CrossValidation): {pipeline.score(X_val, y_val):%}')


Validation Accuracy(CrossValidation): 81.018519%


In [ ]:
y_pred = pipeline.predict(X_test)
y_pred

In [ ]:
submission = test[['id']].copy()
submission['status_group'] = y_pred
submission